## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 


## pre-requisites

- unzip
- awscli
- aws account with a user, don't use root user access

> [!CAUTION]
>
> FIXME below section still needs doing, what I've tried have all failed. Need to read up on permissions
> ### create IAM user
>
> After you have succeefully create an AWS account, login as Root user 
>
> - instead of the root user, designate an IAM user for the EC2 instance: zoomcamp
> - in Security Credentials tab:
>   - assign console access
>   - create Access key ID, Secret access key

### install aws cli

- In local terminal, activate your venv if haven't done so
  - [install or update to the latest version of the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)


```bash
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
```

- check path and version to verify successful install, ensure it is in your venv (as mine is) or in your global `/usr/bin/aws` (if venv not activated before, means install is for all projects)

```bash
 which aws
/home/ellacharmed/github/mlops-zoomcamp/.micromamba/envs/mlops/bin/aws

 aws --version
aws-cli/2.15.58 Python/3.11.8 Linux/5.15.146.1-microsoft-standard-WSL2 exe/x86_64.ubuntu.22
```

- you can then remove the zip file and aws folder after installation is done

- ~~login with the IAM user created before (zoomcamp),~~ and go to [](../mlflow_on_aws.md) for instructions on creating and instantiating
  - EC2 instance to have our mlflow UI on
  - a postgres database in RDS for the backend store
  - an S3 bucket for the artifact store

### launch the tracking server

Connect to EC2 from local wsl+ubuntu. The *.pem is the ssh we setup and the string after the `'@'` is your EC2 instance's Public IPv4 DNS or your Private IPv4 addresses `172-31-39-221`

```bash
chmod 400 ~/.ssh/mlflow-aws.pem
ssh -i "~/.ssh/mlflow-aws.pem" ubuntu@ec2-54-255-172-18.ap-southeast-1.compute.amazonaws.com
```

In the `ec2 terminal`, run the installation steps below
```bash
ubuntu@ip-172-31-39-221:~$
```

**installation steps on ec2 terminal**

```bash
sudo apt update -y && sudo apt upgrade -y
sudo apt install python3.11.9
which python
pip install --upgrade pip
which pip

mkdir mlops
cd mlops
sudo pip3 install pipenv
pipenv install awscli mlflow==2.12.2 boto3 psycopg2-binary
pipenv shell

aws configure # input the Access key ID,Secret access key that you download in csv format (or copy)
aws s3 ls 

(mlops) ubuntu@ip-172-31-39-221:~/mlops$ aws s3 ls
2024-05-25 13:55:10 mlflow-ella-artifacts-remote  # when you get the bucket name you had created, this whole process is successful

```

### launch the Backend store

```bash
in ec2 terminal:
export DB_USER=mlflow
export DB_PASSWORD=your_password
export DB_ENDPOINT=mlflow-db.ctoowsu64oko.ap-southeast-1.rds.amazonaws.com
export DB_NAME=mlflow_db
export S3_BUCKET_NAME=mlflow-ella-artifacts-remote


mlflow server -h 0.0.0.0 -p 5000 \
    --backend-store-uri=postgresql://${DB_USER}:${DB_PASSWORD}@${DB_ENDPOINT}:5432/${DB_NAME} \
    --default-artifact-root=s3://${S3_BUCKET_NAME}
```


In [1]:
import mlflow
import os

# os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials
# used `aws configure` in local macine too

TRACKING_SERVER_HOST = "ec2-54-255-172-18.ap-southeast-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-255-172-18.ap-southeast-1.compute.amazonaws.com:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-ella-artifacts-remote/0', creation_time=1716716281601, experiment_id='0', last_update_time=1716716281601, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("scenario-3")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://mlflow-ella-artifacts-remote/1/96b1794834b04d9aae846e5c72fad911/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-ella-artifacts-remote/1', creation_time=1716716355067, experiment_id='1', last_update_time=1716716355067, lifecycle_stage='active', name='scenario-3', tags={}>,
 <Experiment: artifact_location='s3://mlflow-ella-artifacts-remote/0', creation_time=1716716281601, experiment_id='0', last_update_time=1716716281601, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [9]:
client.search_registered_models()

[]

In [10]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/26 17:43:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716716624048, current_stage='None', description='', last_updated_timestamp=1716716624048, name='iris-classifier', run_id='96b1794834b04d9aae846e5c72fad911', run_link='', source='s3://mlflow-ella-artifacts-remote/1/96b1794834b04d9aae846e5c72fad911/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716716622278, description='', last_updated_timestamp=1716716624048, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716716624048, current_stage='None', description='', last_updated_timestamp=1716716624048, name='iris-classifier', run_id='96b1794834b04d9aae846e5c72fad911', run_link='', source='s3://mlflow-ella-artifacts-remote/1/96b1794834b04d9aae846e5c72fad911/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]